In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from pylab import cm
import os
from scipy.spatial.distance import pdist, squareform

%matplotlib inline

In [ ]:
parent_dir = os.path.split(os.getcwd())[0]

In [ ]:
zones = gpd.read_file(parent_dir + '\\Data\\New\\lms_zone_du_new.shp') # LMS Zone data

In [ ]:
road = gpd.read_file(parent_dir + '\\Data\\Weg\\Wegvakken.shp')

In [ ]:
road_width = pd.read_csv(parent_dir + '\\Data\\Weg\\wkd_005-wegbreedte.csv', delimiter=';')

In [ ]:
park_vlak = pd.read_csv(parent_dir + '\\Data\\Weg\\wkd_006-parkvlak.csv', delimiter=';')
park_punt = pd.read_csv(parent_dir + '\\Data\\Weg\\Parkeerpunten.csv', delimiter=';')

In [ ]:
weg_cat = pd.read_csv(parent_dir + '\\Data\\Weg\\wegcategorisering.csv', delimiter=';')

In [ ]:
# road22 = gpd.read_file(parent_dir + '\\Data\\Weg\\Wegvakken22.shp')

In [ ]:
design_data = zones[['ZONE_ID']]

## Assign roads to LMS zones

In [ ]:
f, ax = plt.subplots(1, 1)
f.set_figwidth(12)

road.plot(ax=ax)
zones.plot(ax=ax, edgecolor='red', facecolor='None')


ax.set_xlim(70000, 90000)
ax.set_ylim(430000, 450000)
ax.set_xticks([], [])
ax.set_yticks([], [])
              


In [ ]:
zones.crs ='EPSG:28992'

First, assign each road section to the closest LMS zone.

In [ ]:
zone_road = zones.sjoin_nearest(road, how='right', max_distance=100)

In [ ]:
zone_road.head()

Now do some checks to see if the roads are assigned logically.

In [ ]:
x = 1144 ## Choose LMS zone

In [ ]:
wvk_id = zone_road[zone_road.ZONE_ID == x].WVK_ID

In [ ]:
f, ax = plt.subplots(1, 1)

zones[zones.ZONE_ID == x].plot(ax=ax, edgecolor='red', facecolor='none', linewidth=2)

for id in wvk_id:
    road[road.WVK_ID == id].plot(ax=ax, edgecolor='black')

ax.set_xticks([], [])
ax.set_yticks([], [])

ax.set_title('All road segments belonging to a random LMS zone')

## Road network density

Looking randomly at zome zones, the match looks pretty good. Now calculate the total length of road for each zone.

In [ ]:
road_dens = np.zeros(1406)

for z in range(1, 1407):
    wvk_id = zone_road[zone_road.ZONE_ID == z].WVK_ID # Get all roads for LMS zone

    length = road[road.WVK_ID.isin(wvk_id)].geometry.length.sum() / 1000
    road_dens[z - 1] = np.round(length / (zones[zones.ZONE_ID == z].OPP.iloc[0] / 100), 2)
    

In [ ]:
road_dens

In [ ]:
design_data.loc[:, 'Road_density'] = road_dens

In [ ]:
design_data

In [ ]:
zones['road_dens'] = road_dens

In [ ]:
f, ax = plt.subplots(1, 1)
f.set_figwidth(13)
f.set_figheight(6)

cmap = cm.get_cmap('YlOrRd')

zones.plot(ax=ax, column='road_dens', cmap=cmap, 
         legend=True, legend_kwds={"label": "Road density [km/km^2]"})

ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor('lightskyblue')
ax.set_title('Road density');

## Road width

In [ ]:
road_width.loc[road_width.BREEDTE == 'onbekend', 'BREEDTE'] = np.nan

In [ ]:
# pc4.loc[:, 'postcode4'] = pc4['postcode4'].astype(int)
road_width.loc[:, 'BREEDTE'] = road_width['BREEDTE'].astype(float)

In [ ]:
road_w = np.zeros(1406)

for z in range(1, 1407):
    wvk_id = zone_road[zone_road.ZONE_ID == z].WVK_ID # Get all roads for LMS zone

    w = road_width[road_width.WEGVAK_ID.isin(wvk_id)].BREEDTE
    road_w[z - 1] = w.mean()
    # length = road[road.WVK_ID.isin(wvk_id)].geometry.length.sum() / 1000
    # road_dens[z - 1] = np.round(length / (zones[zones.ZONE_ID == z].OPP.iloc[0] / 100), 2)

In [ ]:
road_w = np.round(road_w, 2)

In [ ]:
zones['road_width'] = road_w

In [ ]:
f, ax = plt.subplots(1, 1)
f.set_figwidth(13)
f.set_figheight(6)

cmap = cm.get_cmap('YlOrRd')

zones.plot(ax=ax, column='road_width', cmap=cmap,
         legend=True, legend_kwds={"label": "Road width [m]"})

ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor('lightskyblue')
ax.set_title('Road width');

In [ ]:
design_data.loc[:, 'Road_width'] = road_w

In [ ]:
design_data

## Biking

#### Get roads from 2022

Combine roads with zone data and check if it seems okay

In [ ]:
zone_road22 = zones.sjoin_nearest(road, how='right', max_distance=100)

In [ ]:
x = 888

wvk_id = zone_road[zone_road.ZONE_ID == x].WVK_ID

In [ ]:
fl_id = weg_cat[(weg_cat.WEGVAK_ID.isin(wvk_id)) &
                    (weg_cat.WEG_CAT.isin(['fietspad', 'voetpad']))].WEGVAK_ID

In [ ]:
f, ax = plt.subplots(1, 1)

zones[zones.ZONE_ID == x].plot(ax=ax, edgecolor='green', facecolor='none', linewidth=2)

for id in wvk_id:
    road[road.WVK_ID == id].plot(ax=ax, edgecolor='black')

for id in fl_id:
    road[road.WVK_ID == id].plot(ax=ax, edgecolor='red')

ax.set_xticks([], [])
ax.set_yticks([], [])
ax.plot([], [], color='red', label='Bike/pedestian path')
ax.plot([], [], color='black', label='Other road segments')

ax.legend()

ax.set_title('All bike and pedestrian paths of a random LMS zone')
    

In [ ]:
road_f = np.zeros(1406)

for z in range(1, 1407):
    wvk_id = zone_road22[zone_road22.ZONE_ID == z].WVK_ID # Get all roads for LMS zone

    fl_id = weg_cat[(weg_cat.WEGVAK_ID.isin(wvk_id)) &
                    (weg_cat.WEG_CAT.isin(['fietspad', 'voetpad']))].WEGVAK_ID

    length_tot = road22[road22.WVK_ID.isin(wvk_id)].geometry.length.sum()
    length_fl = road22[road22.WVK_ID.isin(fl_id)].geometry.length.sum()

    road_f[z - 1] = np.round(length_fl / length_tot * 100, 3)
    
    # if zones[zones.ZONE_ID == z].INWONERS.iloc[0] > 0:
    #     road_c[z - 1] = (c / (zones[zones.ZONE_ID == z].INWONERS.iloc[0]))
    # else:
    #     road_c[z - 1] = np.nan

In [ ]:
length_fl

In [ ]:
road_f

In [ ]:
zones['Bike_perc'] = road_f

In [ ]:
f, ax = plt.subplots(1, 1)
f.set_figwidth(13)
f.set_figheight(6)

cmap = cm.get_cmap('YlOrRd')

zones.plot(ax=ax, column='Bike_perc', cmap=cmap, 
         legend=True, legend_kwds={"label": "Percentage of bike/pedestrian paths"})

ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor('lightskyblue')
ax.set_title('Percentage road that is for bike/pedestrian');

In [ ]:
f, ax = plt.subplots(1, 1)
f.set_figwidth(13)
f.set_figheight(6)

cmap = cm.get_cmap('YlOrRd')

zones.plot(ax=ax, column='Bike_perc', cmap=cmap, vmax=15,
         legend=True, legend_kwds={"label": "Percentage of bike/pedestrian paths"})

ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor('lightskyblue')
ax.set_title('Percentage road that is for bike/pedestrian (capped at 15%)');

In [ ]:
set(weg_cat.WEG_CAT)

In [ ]:
design_data.loc[:, 'Bike_walk_percentage'] = road_f

## Parking

In [ ]:
park_vlak

In [ ]:
road_c = np.zeros(1406)

for z in range(1, 1407):
    wvk_id = zone_road[zone_road.ZONE_ID == z].WVK_ID # Get all roads for LMS zone

    c = park_vlak[park_vlak.WEGVAK_ID.isin(wvk_id)].OPPERVLAK.sum()
    if zones[zones.ZONE_ID == z].INWONERS.iloc[0] > 0:
        road_c[z - 1] = (c / (zones[zones.ZONE_ID == z].INWONERS.iloc[0]))
    else:
        road_c[z - 1] = np.nan
    
    # road_w[z - 1] = w.mean()

In [ ]:
road_c

In [ ]:
zones['road_c'] = road_c

In [ ]:
road_c[road_c == 0]

In [ ]:
zones.road_c

In [ ]:
f, ax = plt.subplots(1, 1)
f.set_figwidth(13)
f.set_figheight(6)

cmap = cm.get_cmap('YlOrRd')

zones.plot(ax=ax, column='road_c', cmap=cmap, vmax=10,
         legend=True, legend_kwds={"label": "Road parking places"})

ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor('lightskyblue')
ax.set_title('Road parking');

In [ ]:
road_c = np.round(road_c, 2)

In [ ]:
road_c[road_c > 40] = np.nan

In [ ]:
plt.hist(road_c)

In [ ]:
design_data.loc[:, 'Road_parking'] = road_c

In [ ]:
design_data

## Save to csv

In [ ]:
design_data.to_csv(parent_dir + '\\Data\\New\\lms_zone_design.csv')